In [1]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5103)
/home/vijin/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224),class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [3]:
def finetune(model, batches):
    model.pop()
    for layer in model.layers: layer.trainable=False
    model.add(Dense(batches.nb_class, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def fit(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample,verbose=2)

In [7]:
def test(model, path, batch_size=8):
        test_batches = get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
        return test_batches, model.predict_generator(test_batches, test_batches.nb_sample)

In [8]:
def get_model(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dense(1000, activation='softmax'))
    model.add(Dense(8, activation='softmax'))

    #if weights_path:
        #model.load_weights(weights_path)

    return model

In [9]:
vgg_19_model = get_model(weights_path='vgg19_weights.h5')

In [10]:
path = "/home/vijin/DeepLearning/Fisheries_Monitoring/"
batch_size = 4

batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

vgg_19_model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
#finetune(vgg_19_model,batches)

Found 3464 images belonging to 8 classes.
Found 313 images belonging to 8 classes.


In [11]:
fit(vgg_19_model,batches, val_batches, nb_epoch=80)
vgg_19_model.save_weights(path+'result/vgg_19_ft1.h5')

fit(vgg_19_model,batches, val_batches, nb_epoch=50)
vgg_19_model.save_weights(path+'result/vgg_19_ft2.h5')

Epoch 1/80
237s - loss: 12.7285 - acc: 0.2093 - val_loss: 15.7576 - val_acc: 0.0224
Epoch 2/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 3/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8606 - val_acc: 0.0160
Epoch 4/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7576 - val_acc: 0.0224
Epoch 5/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 6/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8606 - val_acc: 0.0160
Epoch 7/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 8/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7576 - val_acc: 0.0224
Epoch 9/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 10/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7576 - val_acc: 0.0224
Epoch 11/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8091 - val_acc: 0.0192
Epoch 12/80
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_a

In [ ]:
fit(vgg_19_model,batches, val_batches, nb_epoch=100)
vgg_19_model.save_weights(path+'result/vgg_19_ft3.h5')

Epoch 1/100
233s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8091 - val_acc: 0.0192
Epoch 2/100
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 3/100
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.9121 - val_acc: 0.0128
Epoch 4/100
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7061 - val_acc: 0.0256
Epoch 5/100
234s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7576 - val_acc: 0.0224
Epoch 6/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8091 - val_acc: 0.0192
Epoch 7/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8606 - val_acc: 0.0160
Epoch 8/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.7061 - val_acc: 0.0256
Epoch 9/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8091 - val_acc: 0.0192
Epoch 10/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.8606 - val_acc: 0.0160
Epoch 11/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.6546 - val_acc: 0.0288
Epoch 12/100
235s - loss: 12.7353 - acc: 0.2099 - val_loss: 15.

In [11]:
#testing
vgg_19_model.load_weights(path+'result/vgg_19_ft6.h5')
test_batches, prediction = test(vgg_19_model,path+'test',batch_size=batch_size)

Found 1000 images belonging to 1 classes.


In [12]:
import re
files = [re.match(r'unknown\/(.*)',x).group(1) for x in test_batches.filenames]
prediction_str = []
for x in prediction:
    prediction_str.append(",".join(format(y, "1.10f") for y in x))

output_map = zip(files , prediction_str)
open(path+'result/submission_NCFM_VGG19_6.csv', 'w').write('\n'.join('%s,%s' % x for x in output_map))